In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np

In [2]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_50d0f5e7b7814eb28b63843965d64dc8 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_50d0f5e7b7814eb28b63843965d64dc8 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_50d0f5e7b7814eb28b63843965d64dc8 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='OhIY7CB3sEqIM7ZMkfxNItlqxIp3LXOaT6pEbnQfPdtg',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_50d0f5e7b7814eb28b63843965d64dc8)

body = client_50d0f5e7b7814eb28b63843965d64dc8.get_object(Bucket='twittersentimentanalysisusingml-donotdelete-pr-pnqosahuzsdbjv',Key='imdb_reviews.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

i_reviews = pd.read_csv(body)
i_reviews.head()


,Reviews,Sentiment
0,<START this film was just brilliant casting lo...,positive
1,<START big hair big boobs bad music and a gian...,negative
2,<START this has to be one of the worst films o...,negative
3,<START the <UNK> <UNK> at storytelling the tra...,positive
4,<START worst mistake of my life br br i picked...,negative


In [6]:
body = client_50d0f5e7b7814eb28b63843965d64dc8.get_object(Bucket='twittersentimentanalysisusingml-donotdelete-pr-pnqosahuzsdbjv',Key='test_reviews.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

test_reviews = pd.read_csv(body)
test_reviews.head()

,Reviews,Sentiment
0,<START please give this one a miss br br <UNK>...,negative
1,<START this film requires a lot of patience be...,positive
2,<START many animation buffs consider <UNK> <UN...,positive
3,<START i generally love this type of movie how...,negative
4,<START like some other people wrote i'm a die ...,positive


In [7]:
body = client_50d0f5e7b7814eb28b63843965d64dc8.get_object(Bucket='twittersentimentanalysisusingml-donotdelete-pr-pnqosahuzsdbjv',Key='word_indexes.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

word_index = pd.read_csv(body)
word_index.head(n=20)

,Words,Indexes
0,tsukino,52009
1,nunnery,52010
2,sonja,16819
3,vani,63954
4,woods,1411
5,spiders,16118
6,hanging,2348
7,woody,2292
8,trawling,52011
9,hold's,52012


In [8]:
word_index = dict(zip(word_index.Words, word_index.Indexes))

In [9]:
word_index["<PAD>"]=0
word_index["<START"]=1
word_index["<UNK>"]=2
word_index["<UNUSED>"]=3

In [10]:
def review_encoder(text):
    arr=[word_index[word] for word in text]
    return arr

In [11]:
train_data,train_labels=i_reviews['Reviews'],i_reviews['Sentiment']
test_data,test_labels=test_reviews['Reviews'],test_reviews['Sentiment']

In [12]:
train_data = train_data.apply(lambda review:review.split())
test_data = test_data.apply(lambda review:review.split())

In [13]:
train_data=train_data.apply(review_encoder)
test_data=test_data.apply(review_encoder)

In [14]:
def encode_sentiments(sentiment):
    if sentiment=='positive':
        return 1
    else:
        return 0
    
train_labels=train_labels.apply(encode_sentiments)
test_labels=test_labels.apply(encode_sentiments)

In [15]:
train_data=keras.preprocessing.sequence.pad_sequences(train_data, value=word_index["<PAD>"], padding ='post', maxlen=500)
test_data=keras.preprocessing.sequence.pad_sequences(test_data, value=word_index["<PAD>"], padding ='post', maxlen=500)

In [16]:
model=keras.Sequential([keras.layers.Embedding(10000, 16, input_length=500),
                      keras.layers.GlobalAveragePooling1D(),
                      keras.layers.Dense(16,activation='relu'),
                      keras.layers.Dense(1,activation='sigmoid')])

In [17]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
history=model.fit(train_data,train_labels,epochs=30,batch_size=512,validation_data=(test_data,test_labels))

Epoch 1/30
49/49 [==============================] - 4s 74ms/step - loss: 0.6927 - accuracy: 0.5259 - val_loss: 0.6901 - val_accuracy: 0.5721
Epoch 2/30
49/49 [==============================] - 3s 58ms/step - loss: 0.6880 - accuracy: 0.6527 - val_loss: 0.6809 - val_accuracy: 0.6400
Epoch 3/30
49/49 [==============================] - 3s 59ms/step - loss: 0.6739 - accuracy: 0.7010 - val_loss: 0.6559 - val_accuracy: 0.7483
Epoch 4/30
49/49 [==============================] - 3s 55ms/step - loss: 0.6419 - accuracy: 0.7739 - val_loss: 0.6117 - val_accuracy: 0.7829
Epoch 5/30
49/49 [==============================] - 3s 52ms/step - loss: 0.5888 - accuracy: 0.8059 - val_loss: 0.5551 - val_accuracy: 0.8058
Epoch 6/30
49/49 [==============================] - 3s 57ms/step - loss: 0.5255 - accuracy: 0.8263 - val_loss: 0.4981 - val_accuracy: 0.8268
Epoch 7/30
49/49 [==============================] - 3s 56ms/step - loss: 0.4650 - accuracy: 0.8502 - val_loss: 0.4511 - val_accuracy: 0.8381
Epoch 8/30
49

In [19]:
loss,accuracy=model.evaluate(test_data,test_labels)

782/782 [==============================] - 2s 2ms/step - loss: 0.2923 - accuracy: 0.8855


In [20]:
index=np.random.randint(1,1500)
user_review=test_reviews.loc[index]
print(user_review)

Reviews      <START after an astronaut dies in space he is ...
Sentiment                                             negative
Name: 1499, dtype: object


In [ ]:
user_review=test_data[index]
user_review=np.array([user_review])
if (model.predict(user_review) > 0.5).astype("int32"):
    print("positive sentiment")
else:
    print("negative sentiment")